In [1]:
import os, sys
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torch.optim import lr_scheduler
from torchvision import datasets, models, transforms, utils
from PIL import Image
from torchvision import transforms
from torchvision.models import vgg16
import glob

from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

In [2]:
DATA_DIR = os.path.join(os.path.abspath(".."), "../camera trap photos/PROJECT/")

In [3]:
# Attribution: [Code from PyTorch docs](https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html?highlight=transfer%20learning)

IMAGE_LENGTH = 711
IMAGE_WIDTH = 400

data_transforms = {
    "train": transforms.Compose(
        [
            transforms.Resize((IMAGE_WIDTH, IMAGE_LENGTH)),     
            transforms.ToTensor(),
            transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]),            
        ]
    ),
    "valid": transforms.Compose(
        [
            transforms.Resize((IMAGE_WIDTH, IMAGE_LENGTH)),                        
            transforms.ToTensor(),
            transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]),                        
        ]
    ),
}
image_datasets = {
    x: datasets.ImageFolder(os.path.join(DATA_DIR, x), data_transforms[x])
    for x in ["train", "valid"]
}
dataloaders = {
    x: torch.utils.data.DataLoader(
        image_datasets[x], batch_size=24, shuffle=True, num_workers=4
    )
    for x in ["train", "valid"]
}
dataset_sizes = {x: len(image_datasets[x]) for x in ["train", "valid"]}
class_names = image_datasets["train"].classes

In [5]:
def get_features(model, train_loader, valid_loader):
    with torch.no_grad():
        Z_train = torch.empty((0, 1024))
        y_train = torch.empty((0))
        Z_valid = torch.empty((0, 1024))
        y_valid = torch.empty((0))
        for X, y in train_loader:
            Z_train = torch.cat((Z_train, model(X)), dim=0)
            y_train = torch.cat((y_train, y))
        for X, y in valid_loader:
            Z_valid = torch.cat((Z_valid, model(X)), dim=0)
            y_valid = torch.cat((y_valid, y))
    return Z_train.detach(), y_train.detach(), Z_valid.detach(), y_valid.detach()

In [6]:
densenet = models.densenet121(weights="DenseNet121_Weights.IMAGENET1K_V1")
densenet.classifier = nn.Identity()  # remove last "classification" layer

In [7]:
Z_train, y_train, Z_valid, y_valid = get_features(
    densenet, dataloaders["train"], dataloaders["valid"]
)

In [8]:
pipe = make_pipeline(StandardScaler(), LogisticRegression(max_iter=3000, C=10))
pipe.fit(Z_train, y_train)
pipe.score(Z_train.numpy(), y_train.numpy())

1.0

In [9]:
pipe.score(Z_valid.numpy(), y_valid.numpy())

0.55

In [12]:
from sklearn.metrics import confusion_matrix

y_pred = pipe.predict(Z_valid.numpy())
cm = confusion_matrix(y_valid, y_pred, labels=[0, 1, 2])
df_cm = pd.DataFrame(cm, 
                     index=[f'Actual {name}' for name in class_names],
                     columns=[f'Predicted {name}' for name in class_names])
print(df_cm)

                Predicted animals  Predicted nothing  Predicted people
Actual animals                 67                  5                 0
Actual nothing                 15                 18                 2
Actual people                  33                 17                 3
